# Jacobi to Cartesian Coordinates

In [1]:
# Library imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import rebound

# Aliases
keras = tf.keras

In [2]:
# Local imports
from utils import load_vartbl, save_vartbl, plot_style
from tf_utils import gpu_grow_memory
from jacobi import make_data_jacobi, make_dataset_cart_to_jac, make_dataset_jac_to_cart, make_dataset_cart_to_cart
from jacobi import CartesianToJacobi, JacobiToCartesian
from jacobi import make_model_cart_to_jac, make_model_jac_to_cart, make_model_cart_to_cart

In [3]:
# Grow GPU memory (must be first operation in TF)
gpu_grow_memory()

In [4]:
# Plot style 
plot_style()

In [5]:
N = 100
num_body = 3
data = make_data_jacobi(N=N, num_body=num_body)
m = data['m']
q = data['q']
v = data['v']
qj = data['qj']
vj = data['vj']
mu = data['mu']

**Review one example A Matrix as a Sanity Check**

In [6]:
# Cumulative mass
M = np.cumsum(m, axis=-1)
M_tot = keras.layers.Reshape(target_shape=(1,))(M[:, num_body-1])

# Assemble num_body x num_body square matrix converting from q to r
A_shape = (N, num_body, num_body)
A_ = np.zeros(A_shape)
A_[:, 0, :] = m / M_tot
for i in range(1, num_body):
    A_[:, i, 0:i] = -m[:, 0:i] / M[:, i-1:i]
    A_[:, i, i] = 1.0

In [7]:
A_[0]

array([[ 0.98923701,  0.00788368,  0.00287931],
       [-1.        ,  1.        ,  0.        ],
       [-0.9920935 , -0.00790645,  1.        ]])

In [8]:
np.sum(A_[0], axis=1)

array([1.00000000e+00, 0.00000000e+00, 5.12227416e-08])

**Create a Dataset**

In [9]:
# Create a tensorflow Dataset instance in both directions
N = 1024
num_body = 3
batch_size = 64

ds_c2j = make_dataset_cart_to_jac(N=N, num_body=num_body, batch_size=batch_size)
ds_j2c = make_dataset_jac_to_cart(N=N, num_body=num_body, batch_size=batch_size)

In [10]:
# Create dataset for autoencoder Cartesian to Cartesian
ds_c2c = make_dataset_cart_to_cart(N=N, num_body=num_body, batch_size=batch_size)

In [11]:
# Example batch - cartesian to jacobi
cart, jac = list(ds_c2j.take(1))[0]

In [12]:
# Unpack tensors
m_ = cart['m']
q_ = cart['q']
v_ = cart['v']
qj_ = jac['qj']
vj_ = jac['vj']
mu_ = jac['mu']

# Review shapes
print(f'Example batch sizes:')
print(f'm  = {m_.shape}')
print(f'q  = {q_.shape}')
print(f'v  = {v_.shape}')
print(f'qj = {qj_.shape}')
print(f'vj = {vj_.shape}')
print(f'mu = {mu_.shape}')

Example batch sizes:
m  = (64, 3)
q  = (64, 3, 3)
v  = (64, 3, 3)
qj = (64, 3, 3)
vj = (64, 3, 3)
mu = (64, 3)


### Check the Conversion Against the Dataset Using Layers

In [13]:
def rms(x):
    return np.sqrt(np.mean(np.square(x)))

In [14]:
# Compute Jacobi coordinates from Cartesian
qj_calc, vj_calc, mu_calc = CartesianToJacobi()([m, q, v])
qj_calc = qj_calc.numpy()
vj_calc = vj_calc.numpy()

# Error vector
err_qj = qj_calc - qj
err_vj = vj_calc - vj

# RMS Errors
rms_qj = rms(err_qj)
rms_vj = rms(err_vj)

# Display RMS errors
print(f'RMS Error qj: {rms_qj:5.3e}')
print(f'RMS Error vj: {rms_vj:5.3e}')

RMS Error qj: 3.741e-07
RMS Error vj: 5.850e-08


In [15]:
# Compute Cartesian coordinates from Jacobi
q_calc, v_calc = JacobiToCartesian()([m, qj, vj])
q_calc = q_calc.numpy()
v_calc = v_calc.numpy()

# Error vector
err_q = q_calc - q
err_v = v_calc - v

# RMS Errors
rms_q = rms(err_q)
rms_v = rms(err_v)

# Display RMS errors
print(f'RMS Error q: {rms_q:5.3e}')
print(f'RMS Error v: {rms_v:5.3e}')

RMS Error q: 5.416e-07
RMS Error v: 6.395e-08


In [16]:
# Compute Cartesian coordinates from autoencoder
q_ae, v_ae = JacobiToCartesian()([m, qj_calc, vj_calc])
q_ae = q_ae.numpy()
v_ae = v_ae.numpy()

# Error vector
err_q_ae = q_ae - q
err_v_ae = v_ae - v

# RMS Errors
rms_q_ae = rms(err_q_ae)
rms_v_ae = rms(err_v_ae)

# Display RMS errors
print(f'RMS Error q: {rms_q_ae:5.3e}')
print(f'RMS Error v: {rms_v_ae:5.3e}')

RMS Error q: 4.691e-07
RMS Error v: 6.442e-08


***Conversion in Both Directions is Very Accurate; so is autoencoder***

### Check the Conversion Against the Dataset Using Models

***Cartesian to Jacobi***

In [17]:
# Build the model
model_c2j = make_model_cart_to_jac(num_body=3)

In [18]:
# model_c2j.summary()

In [19]:
# Inputs to compile the c2j model
optimizer = keras.optimizers.Adam(learning_rate=1.0E-3)

loss = {'qj': keras.losses.MeanSquaredError(),
        'vj': keras.losses.MeanSquaredError(),
        'mu': keras.losses.MeanSquaredError(),
       }

metrics = None

loss_weights = {'qj': 1.0,
                'vj': 1.0,
                'mu': 1.0}

In [20]:
# Compile the c2j model
model_c2j.compile(optimizer=optimizer, loss=loss, metrics=metrics, loss_weights=loss_weights)

In [21]:
model_c2j.evaluate(ds_c2j)

16/16 [==============================] - 0s 18ms/step - loss: 5.0109e-12 - qj_loss: 1.2987e-13 - vj_loss: 1.9483e-15 - mu_loss: 4.8791e-12


[5.010881629335245e-12, 1.2987303e-13, 1.9482954e-15, 4.8790603e-12]

In [22]:
# Invoke model manually with numpy arrays
qj_calc, vj_calc, mu = model_c2j([m, q, v])

# Error vector
err_qj = qj_calc - qj
err_vj = vj_calc - vj

# RMS Errors
rms_qj = rms(err_qj)
rms_vj = rms(err_vj)

# Display RMS errors
print(f'RMS Error qj: {rms_qj:5.3e}')
print(f'RMS Error vj: {rms_vj:5.3e}')

RMS Error qj: 3.741e-07
RMS Error vj: 5.850e-08


In [23]:
print(f'Output shapes:')
print(f'qj: {qj.shape}')
print(f'vj: {vj.shape}')
print(f'mu: {mu.shape}')

Output shapes:
qj: (100, 3, 3)
vj: (100, 3, 3)
mu: (100, 3)


***Jacobi to Cartesian***

In [24]:
# Build the model
model_j2c = make_model_jac_to_cart(num_body=3)

In [25]:
# Inputs to compile the c2e model
loss_j2c = {'q': keras.losses.MeanSquaredError(),
            'v': keras.losses.MeanSquaredError()}

loss_weights_j2c = {'q': 1.0,
                    'v': 1.0}

In [26]:
model_j2c.compile(optimizer=optimizer, loss=loss_j2c, metrics=metrics, loss_weights=loss_weights_j2c)

In [27]:
model_j2c.evaluate(ds_j2c)

16/16 [==============================] - 0s 11ms/step - loss: 2.5768e-13 - q_loss: 2.5328e-13 - v_loss: 4.4073e-15


[2.57683740644487e-13, 2.5327643e-13, 4.4073267e-15]

### Test the Autoencoder on Cartesian Coordinates

In [28]:
model_c2c = make_model_cart_to_cart(num_body=3)

In [29]:
# Inputs to compile the c2e model
loss_c2c = {'q_calc': keras.losses.MeanSquaredError(),
            'v_calc': keras.losses.MeanSquaredError(),}

loss_weights_c2c = {'q_calc': 1.0,
                    'v_calc': 1.0}

In [30]:
model_c2c.compile(optimizer=optimizer, loss=loss_c2c, metrics=metrics, loss_weights=loss_weights_c2c)

In [31]:
model_c2c.evaluate(ds_c2c)

16/16 [==============================] - 0s 13ms/step - loss: 2.1825e-13 - q_calc_loss: 2.1423e-13 - v_calc_loss: 4.0172e-15


[2.182496813279305e-13, 2.1423248e-13, 4.0171876e-15]